In [1]:
using Distributed
addprocs(4)

@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Test
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
    using StatsBase
#     using TraitSimulation, OrdinalMultinomialModels, VarianceComponentModels
end

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1278


In [15]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 1.0

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_b, correct_position = simulate_random_response(xla, k, d, l, Zu=z*intercept);

In [16]:
@time result = fit_iht(Y, xla, z, d=d(), l=l, k=11, max_iter=200)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse linear regression
Link functin = IdentityLink()
Sparsity parameter (k) = 11
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -1577.1707947596879, backtracks = 0, tol = 0.6098646752831619
Iteration 2: loglikelihood = -1484.856813620618, backtracks = 0, tol = 0.1269955771967064
Iteration 3: loglikelihood = -1472.9529635904933, backtracks = 0, tol = 0.05823372413927715
Iteration 4: loglikelihood = -1472.5366421393844, backtracks = 1, tol = 0.004508958581388824
I


IHT estimated 10 nonzero SNP predictors and 1 non-genetic predictors.

Compute time (sec):     0.050909996032714844
Final loglikelihood:    -1472.3905989669602
SNP PVE:                0.8426823000266493
Iterations:             10

Selected genetic predictors:
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      782    -0.437828
   2 │      901     0.747956
   3 │     1204     0.691327
   4 │     1306    -1.42505
   5 │     1655    -0.19456
   6 │     3160    -0.861591
   7 │     3936    -0.147235
   8 │     4201     0.338606
   9 │     4402    -0.126472
  10 │     6879    -1.21895

Selected nongenetic predictors:
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1      1.02016

In [17]:
# compare estimated vs true beta values
[result.beta[correct_position] true_b[correct_position]]

10×2 Array{Float64,2}:
 -0.437828  -0.402269
  0.747956   0.758756
  0.691327   0.729135
 -1.42505   -1.47163
 -0.19456   -0.172668
 -0.861591  -0.847906
  0.338606   0.296183
  0.0       -0.0034339
  0.0        0.125965
 -1.21895   -1.24972

In [21]:
# 1 cores
Random.seed!(2020)
@time mses_new = cv_iht(Y, xla, z, d=d(), l=l, parallel=false);



Crossvalidation Results:
	k	MSE
	1	1218.5008240345846
	2	842.5557378754057
	3	634.0406117147634
	4	487.58369505980653
	5	391.3933763254454
	6	305.31664542373767
	7	267.91347992536646
	8	243.05711924230695
	9	243.47585351013382
	10	245.64651577890663
	11	250.63099201132914
	12	253.98774650724235
	13	254.7946957226353
	14	255.89622437798954
	15	263.60686957526195
	16	269.06640759615067
	17	271.05816592183567
	18	274.4334327761434
	19	279.3301188489498
	20	284.5831155777

Best k = 8

  5.270758 seconds (22.97 M allocations: 519.760 MiB, 1.48% gc time)


In [22]:
# 4 cores
Random.seed!(2020)
@time mses_new = cv_iht(Y, xla, z, d=d(), l=l, parallel=true);



Crossvalidation Results:
	k	MSE
	1	1218.5008240345846
	2	842.5557378754057
	3	634.0406117147634
	4	487.58369505980653
	5	391.3933763254454
	6	305.31664542373767
	7	267.91347992536646
	8	243.05711924230695
	9	243.47585351013382
	10	245.64651577890663
	11	250.63099201132914
	12	253.98774650724235
	13	254.7946957226353
	14	255.89622437798954
	15	263.60686957526195
	16	269.06640759615067
	17	271.05816592183567
	18	274.4334327761434
	19	279.3301188489498
	20	284.5831155777

Best k = 8

  1.756494 seconds (16.96 k allocations: 6.124 MiB)


In [7]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray("multivariate_$(r)traits.bed", n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n, 1)
intercepts = [10.0 1.0] # each trait have different intercept

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=2);

Yt = Matrix(Y')
Zt = Matrix(z')
ktrue = k + count(!iszero, intercepts)
@time result = fit_iht(Yt, Transpose(xla), Zt, k=ktrue, verbose=true)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 12
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = 215.4892687838203, backtracks = 0, tol = 0.1243455904380314
Iteration 2: loglikelihood = 1382.37242885485, backtracks = 0, tol = 0.027019894208789006
Iteration 3: loglikelihood = 1477.7383135165255, backtracks = 0, tol = 0.014225517157910452
Iteration 4: loglikelihood = 1511.714843337414, backtracks = 0, tol = 0.004456457


Compute time (sec):     1.4965541362762451
Final loglikelihood:    1521.6067421001371
Iterations:             15
Trait 1's SNP PVE:      0.554527358091918
Trait 2's SNP PVE:      0.6195879626449299

Trait 1: IHT estimated 4 nonzero SNP predictors
4×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     1197     0.121446
   2 │     5651    -0.200705
   3 │     5797    -1.09767
   4 │     8087     1.2791

Trait 1: IHT estimated 1 non-genetic predictors
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1       10.027

Trait 2: IHT estimated 6 nonzero SNP predictors
6×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      326     0.331882
   2 │     2110     0.575645
   3 │     5375     1.19357
   4 │     5797     0.502072
   5 │     6015     0.81844
   6 │     6813     1.36932

Trait 2: IHT estimated 1 non

In [8]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, true_b[:, 1])
[β1[true_b1_idx] true_b[true_b1_idx, 1]]

4×2 Array{Float64,2}:
 -0.200705  -0.224675
 -1.09767   -1.14044
  0.0       -0.14698
  1.2791     1.25668

In [9]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, true_b[:, 2])
[β2[true_b2_idx] true_b[true_b2_idx, 2]]

6×2 Array{Float64,2}:
 0.331882  0.315219
 0.575645  0.609812
 1.19357   1.20121
 0.502072  0.531549
 0.81844   0.808327
 1.36932   1.43455

In [10]:
# non genetic covariates
[result.c intercepts']

2×2 Array{Float64,2}:
 10.027    10.0
  1.03625   1.0

In [6]:
# covariance matrix
[vec(result.Σ) vec(true_Σ)]

4×2 Array{Float64,2}:
  2.4681    2.53934
 -1.83681  -1.85399
 -1.83681  -1.85399
  2.42177   2.41416

In [29]:
# 1 core
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, Transpose(xla), Zt, path=1:20, parallel=false);



Crossvalidation Results:
	k	MSE
	1	2888.7160633632484
	2	2560.135862053543
	3	2067.943067029389
	4	1812.0395079444284
	5	1554.3120367449362
	6	1277.3237598020087
	7	1154.9320629872832
	8	1098.5910963871872
	9	1019.4597637296985
	10	1030.1412464715597
	11	1023.5545874904792
	12	1007.9022110997687
	13	1012.6193656356761
	14	1019.1491606608182
	15	1024.6877890077092
	16	1022.9300595671257
	17	1040.0286509856787
	18	1033.345570850089
	19	1039.8828186471897
	20	1036.274158344765

Best k = 12

223.626565 seconds (5.55 M allocations: 654.873 MiB, 0.08% gc time)


In [13]:
# 4 core
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, Transpose(xla), Zt, path=1:20, parallel=true);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:43




Crossvalidation Results:
	k	MSE
	1	2888.7160633632484
	2	2560.135862053543
	3	2067.943067029389
	4	1812.0395079444284
	5	1554.3120367449362
	6	1277.3237598020087
	7	1154.9320629872832
	8	1098.5910963871872
	9	1019.4597637296985
	10	1030.1412464715597
	11	1023.5545874904792
	12	1007.9022110997687
	13	1012.6193656356761
	14	1019.1491606608182
	15	1024.6877890077092
	16	1022.9300595671257
	17	1040.0286509856787
	18	1033.345570850089
	19	1039.8828186471897
	20	1036.274158344765

Best k = 12

 43.694680 seconds (43.40 k allocations: 8.408 MiB)


# First simulate multivariate Gaussian traits


With $r$ traits, each sample's phenotype $\mathbf{y}_{i}$ is simulated under

$$\mathbf{y}_{i}^{r \times 1} \sim N(\mathbf{B}^{r \times p}\mathbf{x}_{i}^{p \times 1}, \ \ \Sigma_{r \times r})$$

In [2]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray("multivariate_$(r)traits.bed", n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n, 1)
intercepts = [10.0 1.0] # each trait have different intercept

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=2);

In [5]:
# save true SNP's position and effect size
open("multivariate_$(r)traits_true_beta.txt", "w") do io
    println(io, "snpID,effectsize")
    for pos in correct_position
        println(io, "snp$pos,", true_b[pos])
    end
end

# create `.bim` and `.bam` files using phenotype
make_bim_fam_files(x, Y, "multivariate_$(r)traits")

# save phenotypes in separate file
open("multivariate_$(r)traits.phen", "w") do io
    println(io, "FID\tIID\tT1\tT2")
    for i in 1:n
        println(io, "$i\t1\t", Y[i, 1], "\t", Y[i, 2])
    end
end

# Run IHT

In [7]:
Yt = Matrix(Y')
Zt = Matrix(z')
ktrue = k + count(!iszero, intercepts)
@time result = fit_iht(Yt, Transpose(xla), Zt, k=11, verbose=true)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 11
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 100
Converging when tol < 0.0001:

Iteration 1: loglikelihood = 215.4892687838203, backtracks = 0, tol = 0.1258803451727764
Iteration 2: loglikelihood = 1400.4415131326584, backtracks = 0, tol = 0.027353391443577676
Iteration 3: loglikelihood = 1485.32487407949, backtracks = 0, tol = 0.007379616806130262
Iteration 4: loglikelihood = 1498.0290279587853, backtracks = 0, tol = 0.01194219


Compute time (sec):     1.2903869152069092
Final loglikelihood:    1514.818382728515
Iterations:             12
Trait 1's SNP PVE:      0.5517175992313121
Trait 2's SNP PVE:      0.6190809967368884

Trait 1: IHT estimated 3 nonzero SNP predictors
3×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     5651    -0.200754
   2 │     5797    -1.09966
   3 │     8087     1.27901

Trait 1: IHT estimated 1 non-genetic predictors
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1       10.027

Trait 2: IHT estimated 6 nonzero SNP predictors
6×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      326     0.331488
   2 │     2110     0.574476
   3 │     5375     1.19453
   4 │     5797     0.501675
   5 │     6015     0.81899
   6 │     6813     1.36762

Trait 2: IHT estimated 1 non-genetic predictors
1×2 Data

## Check answer

In [8]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, true_b[:, 1])
[β1[true_b1_idx] true_b[true_b1_idx, 1]]

4×2 Array{Float64,2}:
 -0.200754  -0.224675
 -1.09966   -1.14044
  0.0       -0.14698
  1.27901    1.25668

In [9]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, true_b[:, 2])
[β2[true_b2_idx] true_b[true_b2_idx, 2]]

6×2 Array{Float64,2}:
 0.331488  0.315219
 0.574476  0.609812
 1.19453   1.20121
 0.501675  0.531549
 0.81899   0.808327
 1.36762   1.43455

In [10]:
# non genetic covariates
[result.c intercepts']

2×2 Array{Float64,2}:
 10.027    10.0
  1.03625   1.0

In [11]:
# covariance matrix
[vec(result.Σ) vec(true_Σ)]

4×2 Array{Float64,2}:
  2.48446   2.53934
 -1.83826  -1.85399
 -1.83826  -1.85399
  2.42229   2.41416

In [12]:
# number of causal SNPs recovered
correct_snps = [x[1] for x in correct_position]  # truely causal snps
signif_snps = true_b1_idx ∪ true_b2_idx          # IHT's selected snps
signif_snps ∩ correct_snps

8-element Array{Int64,1}:
 5651
 5797
 6813
 8087
  326
 2110
 5375
 6015

# Test Cross validation

In [36]:
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, x, Zt);

LoadError: [91mMethodError: no method matching cv_iht(::Array{Float64,2}, ::SnpArray, ::Array{Float64,2})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  cv_iht(::Union{AbstractArray{T,1}, AbstractArray{T,2}}, [91m::AbstractArray{T,2}[39m, ::Union{AbstractArray{T,1}, AbstractArray{T,2}}; d, l, path, q, est_r, group, weight, folds, destin, use_maf, debias, verbose, parallel, max_iter) where T<:Union{Float32, Float64} at /Users/biona001/.julia/dev/MendelIHT/src/cross_validation.jl:33[39m
[91m[0m  cv_iht(::Union{AbstractArray{T,1}, AbstractArray{T,2}}, ::AbstractArray{T,2} where T; kwargs...) where T at /Users/biona001/.julia/dev/MendelIHT/src/cross_validation.jl:93[39m

In [15]:
argmin(mses)

11

**Conclusion:** After cross validation, IHT finds 10/12 predictors and 8/10 causal SNPs. 

# GEMMA multivariate results

In [9]:
gemma_df = CSV.read("gemma.result.assoc.txt", DataFrame)

# pvalues
pval_wald = gemma_df[!, :p_wald]
pval_lrt = gemma_df[!, :p_lrt]
pval_score = gemma_df[!, :p_score]

# estimated beta
estim_β1 = gemma_df[!, :beta_1]
estim_β2 = gemma_df[!, :beta_2]

# estimated covariance matrix
estim_σ11 = gemma_df[!, :Vbeta_1_1]
estim_σ12 = gemma_df[!, :Vbeta_1_2]
estim_σ22 = gemma_df[!, :Vbeta_2_2];

In [10]:
correct_snps = [x[1] for x in correct_position]  # truely causal snps
signif_snps = findall(x -> x ≤ 0.05/p, pval_lrt) # gemma's selected snps
signif_snps ∩ correct_snps

6-element Array{Int64,1}:
 2110
 5375
 5797
 6015
 6813
 8087

**Conclusion:** GEMMA finds 6/10 causal SNPs

# MV-PLINK

In [25]:
mvplink_df = CSV.read("plink.mqfam.total", DataFrame, delim=' ', ignorerepeated=true)

# pvalues
pval = mvplink_df[!, :P]

# SNPs passing threshold
signif_snps = findall(x -> x ≤ 0.05 / p, pval)
signif_snps ∩ correct_snps

6-element Array{Int64,1}:
 2110
 5375
 5797
 6015
 6813
 8087

**Conclusion:** MV-PLINK finds 6/10 causal SNPs

# Compare with univariate IHT

+ `fit_iht`: ~20 times slower
+ `cv_iht`: ~70 times slower

In [49]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 1.0

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_b, correct_position = simulate_random_response(xla, k, d, l, Zu=z*intercept);

In [50]:
@time result = fit_iht(Y, xla, z, d=d(), l=l, k=11)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse linear regression
Link functin = IdentityLink()
Sparsity parameter (k) = 11
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Debias = off
Max IHT iterations = 100
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -1577.170794759688, backtracks = 0, tol = 0.609864675283163
Iteration 2: loglikelihood = -1484.8568136206177, backtracks = 0, tol = 0.1269955771967065
Iteration 3: loglikelihood = -1472.9529635904933, backtracks = 0, tol = 0.05823372413927707
Iteration 4: loglikelihood = -1472.5366421393842, backtracks = 1, tol = 0.00450895


IHT estimated 10 nonzero SNP predictors and 1 non-genetic predictors.

Compute time (sec):     0.06683206558227539
Final loglikelihood:    -1472.3905989669602
Iterations:             10

Selected genetic predictors:
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      782    -0.437828
   2 │      901     0.747956
   3 │     1204     0.691327
   4 │     1306    -1.42505
   5 │     1655    -0.19456
   6 │     3160    -0.861591
   7 │     3936    -0.147235
   8 │     4201     0.338606
   9 │     4402    -0.126472
  10 │     6879    -1.21895

Selected nongenetic predictors:
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1      1.02016

In [41]:
# compare estimated vs true beta values
[result.beta[correct_position] true_b[correct_position]]

10×2 Array{Float64,2}:
 -0.437828  -0.402269
  0.747956   0.758756
  0.691327   0.729135
 -1.42505   -1.47163
 -0.19456   -0.172668
 -0.861591  -0.847906
  0.338606   0.296183
  0.0       -0.0034339
  0.0        0.125965
 -1.21895   -1.24972

### Cross validation timing

In [92]:
# 1 cores
Random.seed!(2020)
@time cv_iht(Y, x, z);



Crossvalidation Results:
	k	MSE
	1	1221.686764265696
	2	864.1553085156108
	3	661.6430376327284
	4	516.2679723147669
	5	418.2721191658746
	6	342.7931325462592
	7	303.15938053956916
	8	276.6241892508097
	9	278.29180795455045
	10	280.31878745464377
	11	282.10757732881564
	12	285.1294156734012
	13	293.16875008847467
	14	297.43739905389987
	15	296.17300065041906
	16	307.7814278347698
	17	302.0113818893564
	18	312.70262374274137
	19	309.87367962700966
	20	315.6750311115386
  4.315330 seconds (9.35 M allocations: 332.277 MiB, 1.12% gc time)


# More complicated univariate traits

Let's simulate $10$ causal SNPs where genetics explain 60% of heritability for $2$ traits. The univariate model is:

$$\mathbf{y}_{n \times 1} \sim N(\mathbf{X}_{n \times p}\mathbf{\beta}_{p \times 1} \ , \ \sigma_g^2 \Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n}).$$

where $\Phi$ is the GRM and $\sigma_g^2 = 0.6, \sigma_e^2 = 0.4$. 

In [3]:
"""
k = Number of causal SNPs
p = Total number of SNPs
traits = Number of traits (phenotypes)
overlap = number of causal SNPs shared in each trait
"""
function simulate_random_beta(k::Int, p::Int, traits::Int; overlap::Int=0)
    true_b = zeros(p, traits)
    if overlap == 0
        causal_snps = sample(1:(traits * p), k, replace=false)
        true_b[causal_snps] = randn(k)
    else
        shared_snps = sample(1:p, overlap, replace=false)
        weight_vector = aweights(1 / (traits * (p - overlap)) * ones(traits * p))
        for i in 1:traits
            weight_vector[i*shared_snps] .= 0.0 # avoid sampling from shared snps
        end
        @assert sum(weight_vector) ≈ 1.0
        # simulate β for shared predictors
        for i in 1:traits
            true_b[shared_snps, i] = randn(overlap)
        end
        # simulate β for none shared predictors
        nonshared_snps = sample(1:(traits * p), weight_vector, k - traits * overlap, replace=false)
        true_b[nonshared_snps] = randn(k - traits * overlap)
    end

    return true_b
end

simulate_random_beta

In [267]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# simulate true_b with 1 phenotype
b = simulate_random_beta(k, p, 1)

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 1.0

# between sample covariance is identity + GRM (2 times because in SnpArrays grm is halved)
Φ = 2grm(x)
I_n = Matrix{Float64}(I, size(Φ))
σg = 0.6
σe = 0.4
V = σg * Φ + σe * I_n;

# simulate response y
mvn = MvNormal(vec(intercept * z + xla * b), V)
y = rand(mvn);

In [268]:
ktrue = k + count(!iszero, intercept)
@time result = fit_iht(y, xla, z, k=ktrue, max_iter=200)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse linear regression
Link functin = IdentityLink()
Sparsity parameter (k) = 11
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -1627.0353462544151, backtracks = 0, tol = 0.6924353417126095
Iteration 2: loglikelihood = -1420.1638948810214, backtracks = 0, tol = 0.23769360612297397
Iteration 3: loglikelihood = -1406.552555178033, backtracks = 0, tol = 0.07456091670768471
Iteration 4: loglikelihood = -1405.9717536588919, backtracks = 0, tol = 0.007850378616075855



IHT estimated 10 nonzero SNP predictors and 1 non-genetic predictors.

Compute time (sec):     0.034310102462768555
Final loglikelihood:    -1405.9651489743871
SNP PVE:                0.9024422743420005
Iterations:             7

Selected genetic predictors:
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      329     0.501429
   2 │     1438     0.852922
   3 │     5797    -1.1783
   4 │     6307     1.5088
   5 │     6813    -0.158366
   6 │     7152     1.07684
   7 │     8070     0.772066
   8 │     8739     1.38536
   9 │     9138    -0.586787
  10 │     9180     0.563219

Selected nongenetic predictors:
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1      1.01532

In [271]:
# compare estimated vs true beta values
correct_position = findall(!iszero, b)
[result.beta[correct_position] b[correct_position]]

10×2 Array{Float64,2}:
  0.501429   0.531549
  0.852922   0.882913
 -1.1783    -1.14044
  1.5088     1.43455
 -0.158366  -0.14698
  1.07684    1.11302
  0.772066   0.812423
  1.38536    1.37084
 -0.586787  -0.589568
  0.563219   0.622075

# More complicated multivariate traits

With $r$ traits, the multivariate analog is:

$$\mathbf{Y}_{r \times n} \sim \text{MatrixNormal}(\mathbf{B}_{r \times p}\mathbf{X}_{p \times n}, \ \ \Sigma_{r \times r} , \ \ \sigma_g^2\Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n \times n})$$

where $\Sigma_{r \times r}$ is the row covariance matrix, and $\sigma_g^2\Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n \times n}$ is the column covariance matrix. Again, $\Phi$ is the GRM and $\sigma_g^2 = 0.6, \sigma_e^2 = 0.4$. 

In [6]:
n = 5000  # number of samples
p = 300000 # number of SNPs
k = 20    # number of causal SNPs
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray("multivariate_$(r)traits.bed", n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n, 1)
intercepts = [10.0 1.0] # each trait have different intercept

# simulate β
B = simulate_random_beta(k, p, r, overlap=2)

# between trait covariance matrix
Σ = random_covariance_matrix(r)

# between sample covariance is identity + GRM (2 times because in SnpArrays grm is halved)
Φ = 2grm(x)
σg = 0.6
σe = 0.4
V = σg * Φ + σe * I

# simulate y using TraitSimulations.jl
# VCM_model = VCMTrait(z, intercepts, x, B, [Σ], [V]) #https://github.com/OpenMendel/TraitSimulation.jl/blob/6d1f09c7332471a74b4dd6c8ef2d2b95a96c585c/src/modelframework.jl#L159
# Y = simulate(VCM_model)

# simulate using naive model
μ = z * intercepts + xla * B
Yt = rand(MatrixNormal(μ', Σ, V))

2×5000 Array{Float64,2}:
 9.76395  10.4957    6.63228   7.96867  …  14.73     13.2039   6.39848
 3.71778   2.88821  -0.811218  4.21819      4.05212  -1.76484  3.48013

## Run IHT 

In [7]:
# Yt = Matrix(Y')
Zt = Matrix(z')
ktrue = k + count(!iszero, intercepts)
@time result = fit_iht(Yt, Transpose(xla), Zt, k=ktrue, verbose=true)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 22
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -1264.131527480402, backtracks = 0, tol = 0.1692812087308949
Iteration 2: loglikelihood = 10249.41368144947, backtracks = 0, tol = 0.020764943950580667
Iteration 3: loglikelihood = 11674.692814587752, backtracks = 0, tol = 0.004013689748612351
Iteration 4: loglikelihood = 11915.96174083871, backtracks = 0, tol = 0.0020765


Compute time (sec):     184.65532207489014
Final loglikelihood:    12046.664460271588
Iterations:             14
Trait 1's SNP PVE:      0.7190577110805266
Trait 2's SNP PVE:      0.8757880134209874

Trait 1: IHT estimated 8 nonzero SNP predictors
8×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │    19535    0.431495
   2 │    27695    0.160205
   3 │    42127    1.67692
   4 │    94323    1.16401
   5 │   123845   -0.667388
   6 │   135962    0.254199
   7 │   199652    0.0555613
   8 │   282837   -0.565194

Trait 1: IHT estimated 1 non-genetic predictors
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1      9.99063

Trait 2: IHT estimated 12 nonzero SNP predictors
12×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     7207    0.5795
   2 │    19535   -1.04121
   3 │    44373    0.631681
   4 │

In [8]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, B[:, 1])
[β1[true_b1_idx] B[true_b1_idx, 1]]

8×2 Array{Float64,2}:
  0.431495    0.437589
  0.160205    0.159139
  1.67692     1.67961
  1.16401     1.16346
 -0.667388   -0.661881
  0.254199    0.26366
  0.0555613   0.0649943
 -0.565194   -0.567334

In [9]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, B[:, 2])
[β2[true_b2_idx] B[true_b2_idx, 2]]

12×2 Array{Float64,2}:
  0.5795      0.582386
 -1.04121    -1.03979
  0.0         0.00530093
  0.631681    0.635882
  0.262662    0.265646
 -1.24585    -1.23606
 -1.63816    -1.63252
 -0.189445   -0.177421
  1.92502     1.92917
  0.563273    0.563348
 -0.635568   -0.641005
 -0.0335199  -0.0251614

In [10]:
# non genetic covariates
[result.c intercepts']

2×2 Array{Float64,2}:
 9.99063   10.0
 0.993229   1.0

In [11]:
# covariance matrix
[vec(result.Σ) vec(Σ)]

4×2 Array{Float64,2}:
 2.00157  1.96481
 1.68337  1.65099
 1.68337  1.65099
 1.63609  1.61449

In [ ]:
# 8 core = 1.5h for 5k by 300k matrix
Random.seed!(2020)
Zt = Matrix(z')
@time mses = cv_iht(Yt, Transpose(xla), Zt, path=1:20, parallel=true);

Progress:   9%|███▊                                     |  ETA: 1:21:28